In [2]:
import json
import pandas as pd
import numpy as np
import networkx as nx
import jellyfish
import os
import shutil
import subprocess
from scipy.cluster.hierarchy import dendrogram, linkage
from matplotlib import pyplot as plt
from fuzzywuzzy import fuzz
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.metrics import adjusted_rand_score
from sklearn import preprocessing
from sklearn.cluster import AgglomerativeClustering
from zipfile import ZipFile
from filecmp import dircmp

C:\ProgramData\Anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [1]:
project_name = 'apache_maven'

In [36]:
n_clusters = [50,100,150,200,250,300]
affinity = ['euclidean', 'l1', 'l2', 'manhattan', 'cosine']
linkage = ['complete', 'average', 'single']


### Uncomment below for testing purposes ###
#n_clusters = [50]
#affinity = ['euclidean']
#linkage = ['complete']


def alvin_god(user_n_cluster, user_affinity, user_linkage, dir_arr, depends_arr, file_version, main_dict):
    vectorizer = TfidfVectorizer(stop_words='english')
    X = vectorizer.fit_transform(dir_arr)

    true_k = user_n_cluster
    model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
    model.fit(X)

    #print("Top terms per cluster:")
    order_centroids = model.cluster_centers_.argsort()[:, ::-1]
    terms = vectorizer.get_feature_names()
    
    
    
    
    
    
    main_cluster_arr = []
    col_counter = 0

    
    cluster = AgglomerativeClustering(n_clusters=user_n_cluster, affinity=user_affinity, linkage=user_linkage)
    cluster_result = cluster.fit_predict(depends_arr)

    cluster_result_arr = []
    cluster_counter = 0
    for element in cluster_result:
        #print(element)
        cluster_result_arr.append(element)

    main_cluster_arr.append(cluster_result_arr)
    
    col_counter += 1
    
    dir_arr_int = []
    #for element in dir_arr:
    #    dir_arr_int.append(main_dict[element])
    
    #cluster_result_int = []
    #for element in cluster_result:
        #cluster_result_int.append(main_dict[main_dict_val[element]])
    
    #print(dir_arr_int)
    #print('fk')
    #print(cluster_result_int)
    #common_results_rsf = common_member(dir_arr_int,cluster_result_int)
    #print(len(common_results_rsf))
    
    
    filename1 = 'MoJo_1.2.1/apache_maven/' +str(file_version) + '_' + str(user_n_cluster) + '_' + str(user_affinity) + "_" + str(user_linkage) + '_b' +".rsf"
    #print(filename1)
    with open(filename1, 'w') as f:
        for i in range(len(dir_arr)):
            Y = vectorizer.transform([dir_arr[i]])
            string = "contain " + str(model.predict(Y)[0]) + " " +str(i) + "\n"
            f.write(string)

        f.close()
    
    filename2 = 'MoJo_1.2.1/apache_maven/' +str(file_version) + '_' + str(user_n_cluster) + '_' + str(user_affinity) + "_" + str(user_linkage) + '_a' +".rsf"
    with open(filename2, 'w') as f:
        for i in range(len(cluster_result)):
            #print(element)
            string = "contain " + str(cluster_result[i]) + " " + str(i) + "\n"
            f.write(string)
        f.close()
        #for element in cluster_result:
            #print(element)
            #string = "contain " + str(element) + " " + str(main_dict[main_dict_val[element]]) + "\n"
            #f.write(string)
        #f.close()

    #print('Difference is: ' + str(len(dir_arr) - len(G.nodes) ))
    if len(dir_arr) - len(G.nodes) > 0:
        with open(filename2,'a') as f:
            for i in range(len(dir_arr)-len(G.nodes)):
                tbc = (i+len(G.nodes))
                string = "contain " + str(tbc) + " " + str(tbc) + "\n"
                f.write(string)

            f.close()
    else:
        with open(filename1,'a') as f:
            for i in range(len(G.nodes) - len(dir_arr)):
                tbc = i+len(dir_arr)
                string = "contain " + str(tbc) + " " + str(tbc) + "\n"
                f.write(string)
            f.close()

    return filename1, filename2




def fileIsSame(right, left, path):
    return os.path.exists (os.path.join(left, path.replace(right, '')));

def compare(right, left):
    difference = list();
    for root, dirs, files in os.walk(right):
        for name in files:
            path = os.path.join(root, name);
            # check if file is same
            if fileIsSame(right, left, path):
                if os.path.isdir(path):
                    # recursively check subdirs
                    difference.extend(compare(path, left));
            else:
                # count file as difference
                difference.append(path);

    return difference;

def compare_similar(right, left):
    difference = list();
    for root, dirs, files in os.walk(right):
        for name in files:
            path = os.path.join(root, name);
            # check if file is same
            if fileIsSame(right, left, path):
                
                if os.path.isdir(path):
                    # recursively check subdirs
                    difference.extend(compare_similar(path, left));
                difference.append(path)
                
            #else:
                # count file as difference
                #difference.append(path);

    return difference;



def common_member(a,b):
    a_set = set(a)
    b_set = set(b)
    if len(a_set.intersection(b_set)) > 0:
        return(a_set.intersection(b_set))
    else:
        return set()
    
    


In [8]:
### Run only to unzip source code

rootdir = 'raw_sourcecode_zip'
dir_arr = []
for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        print(os.path.join(subdir, file))
        if 'Cassandra' in os.path.join(subdir, file):
            name = str(os.path.join(subdir, file)).split('\\')
            name = name[1]
            print(name)
            with ZipFile(os.path.join(subdir, file), 'r') as zipObj:
                zipObj.extractall('raw_sourcecode/' + name)
        ### Only look for Java Files ###
        #if ".java" in str(os.path.join(subdir, file)):
            #print(os.path.join(subdir, file))
            #dir_arr.append(str(os.path.join(subdir, file)))
            
print(len(dir_arr))

raw_sourcecode_zip\Apache Cassandra Versions\cassandra_10_3.4.zip
Apache Cassandra Versions


FileNotFoundError: [WinError 206] The filename or extension is too long: 'raw_sourcecode\\Apache Cassandra Versions\\cassandra-cassandra-3.4\\test\\data\\migration-sstables\\2.2\\system\\compactions_in_progress-55080ab05d9c388690a4acb25fe1f77b\\snapshots\\1435298241281-upgrade-3.0.0-SNAPSHOT-2.2.0-rc1-SNAPSHOT'

In [3]:
rootdir = 'raw_sourcecode'
dir_arr = []
depth=2

for root, dirs, files in os.walk(rootdir):
    if root.count(os.sep) == depth and "Apache_Maven_versions" in str(root):
        print(root)
        dir_arr.append(root)
        #for file in files: 
            #print(os.path.join(subdir, file))
        ### Only look for Java Files ###
        #if "Apache Spark versions" in str(os.path.join(subdir, file)):
            #print(os.path.join(subdir, file))
        
            #dir_arr.append(str(os.path.join(subdir, file)))
            
print(len(dir_arr))

raw_sourcecode\Apache_Maven_versions\maven-maven-3.2.0
raw_sourcecode\Apache_Maven_versions\maven-maven-3.2.1
raw_sourcecode\Apache_Maven_versions\maven-maven-3.2.2
raw_sourcecode\Apache_Maven_versions\maven-maven-3.2.3
raw_sourcecode\Apache_Maven_versions\maven-maven-3.2.4
raw_sourcecode\Apache_Maven_versions\maven-maven-3.2.5
raw_sourcecode\Apache_Maven_versions\maven-maven-3.3.0
raw_sourcecode\Apache_Maven_versions\maven-maven-3.3.1
raw_sourcecode\Apache_Maven_versions\maven-maven-3.3.2
raw_sourcecode\Apache_Maven_versions\maven-maven-3.3.3
raw_sourcecode\Apache_Maven_versions\maven-maven-3.3.4
raw_sourcecode\Apache_Maven_versions\maven-maven-3.3.5
raw_sourcecode\Apache_Maven_versions\maven-maven-3.3.6
raw_sourcecode\Apache_Maven_versions\maven-maven-3.3.7
raw_sourcecode\Apache_Maven_versions\maven-maven-3.3.8
raw_sourcecode\Apache_Maven_versions\maven-maven-3.3.9
raw_sourcecode\Apache_Maven_versions\maven-maven-3.5.0
raw_sourcecode\Apache_Maven_versions\maven-maven-3.5.1
raw_source

In [22]:
import os

for i in range(len(dir_arr)):
    #print(element)
    command = 'cd C:/Users/tanji/Desktop/FIT4003/depends-0.9.2 & ' + 'java -jar depends.jar java C:/Users/tanji/Desktop/FIT4003/raw_sourcecode/Apache_Maven_versions/' + dir_arr[i].split('\\')[-1] +  ' ../raw_depends/apache_maven/' + str(i) + '_' + dir_arr[i].split('\\')[-1] 
    print(command)
    #os.system('cd C:/Users/tanji/Desktop/FIT4003/MoJo_1.2.1 & java MoJo test/test2_a.rsf test/test2_b.rsf > test/test_results.txt')
    os.system(command)

cd C:/Users/tanji/Desktop/FIT4003/depends-0.9.2 & java -jar depends.jar java C:/Users/tanji/Desktop/FIT4003/raw_sourcecode/Apache_Maven_versions/maven-maven-3.2.0 ../raw_depends/apache_maven/0_maven-maven-3.2.0
cd C:/Users/tanji/Desktop/FIT4003/depends-0.9.2 & java -jar depends.jar java C:/Users/tanji/Desktop/FIT4003/raw_sourcecode/Apache_Maven_versions/maven-maven-3.2.1 ../raw_depends/apache_maven/1_maven-maven-3.2.1
cd C:/Users/tanji/Desktop/FIT4003/depends-0.9.2 & java -jar depends.jar java C:/Users/tanji/Desktop/FIT4003/raw_sourcecode/Apache_Maven_versions/maven-maven-3.2.2 ../raw_depends/apache_maven/2_maven-maven-3.2.2
cd C:/Users/tanji/Desktop/FIT4003/depends-0.9.2 & java -jar depends.jar java C:/Users/tanji/Desktop/FIT4003/raw_sourcecode/Apache_Maven_versions/maven-maven-3.2.3 ../raw_depends/apache_maven/3_maven-maven-3.2.3
cd C:/Users/tanji/Desktop/FIT4003/depends-0.9.2 & java -jar depends.jar java C:/Users/tanji/Desktop/FIT4003/raw_sourcecode/Apache_Maven_versions/maven-maven

In [23]:
pwd

'C:\\Users\\tanji\\Desktop\\FIT4003'

In [27]:
rootdir = 'raw_depends/apache_maven'
json_name_dict = {}
json_result_dict = {}
for root, dirs, files in os.walk(rootdir):
   
    print(files)
    for element in files:
        
        with open(rootdir + '/' + element) as f:
            tmp = json.load(f)
        
        element = element.split('_')
        json_name_dict[element[0]] = element[1]
        
        
        
        json_result_dict[element[0]] = tmp
        

['0_maven-maven-3.2.0.json', '10_maven-maven-3.3.4.json', '11_maven-maven-3.3.5.json', '12_maven-maven-3.3.6.json', '13_maven-maven-3.3.7.json', '14_maven-maven-3.3.8.json', '15_maven-maven-3.3.9.json', '16_maven-maven-3.5.0.json', '17_maven-maven-3.5.1.json', '18_maven-maven-3.5.2.json', '19_maven-maven-3.5.3.json', '1_maven-maven-3.2.1.json', '20_maven-maven-3.5.4.json', '21_maven-maven-3.6.0.json', '22_maven-maven-3.6.1.json', '23_maven-maven-3.6.2.json', '2_maven-maven-3.2.2.json', '3_maven-maven-3.2.3.json', '4_maven-maven-3.2.4.json', '5_maven-maven-3.2.5.json', '6_maven-maven-3.3.0.json', '7_maven-maven-3.3.1.json', '8_maven-maven-3.3.2.json', '9_maven-maven-3.3.3.json']


In [28]:
json_name_dict

{'0': 'maven-maven-3.2.0.json',
 '10': 'maven-maven-3.3.4.json',
 '11': 'maven-maven-3.3.5.json',
 '12': 'maven-maven-3.3.6.json',
 '13': 'maven-maven-3.3.7.json',
 '14': 'maven-maven-3.3.8.json',
 '15': 'maven-maven-3.3.9.json',
 '16': 'maven-maven-3.5.0.json',
 '17': 'maven-maven-3.5.1.json',
 '18': 'maven-maven-3.5.2.json',
 '19': 'maven-maven-3.5.3.json',
 '1': 'maven-maven-3.2.1.json',
 '20': 'maven-maven-3.5.4.json',
 '21': 'maven-maven-3.6.0.json',
 '22': 'maven-maven-3.6.1.json',
 '23': 'maven-maven-3.6.2.json',
 '2': 'maven-maven-3.2.2.json',
 '3': 'maven-maven-3.2.3.json',
 '4': 'maven-maven-3.2.4.json',
 '5': 'maven-maven-3.2.5.json',
 '6': 'maven-maven-3.3.0.json',
 '7': 'maven-maven-3.3.1.json',
 '8': 'maven-maven-3.3.2.json',
 '9': 'maven-maven-3.3.3.json'}

In [29]:
### Getting the ground truth by comparing previous 10 versions (Burden AF)

ground_truth_dict = {}

root_dir = 'raw_sourcecode/Apache_Maven_versions/'
for i in range(10,20):
    file1 = str(root_dir +json_name_dict[str(i)]).replace('.json', '') + '/'
    for j in range(1,11):
        file2 = str(root_dir + json_name_dict[str(i-j)]).replace('.json', '') + '/'
        
        #test = compare_similar()
        print(file1, file2)
        if (j == 1):
            prev_tmp = compare_similar(file1, file2)
        else:
            tmp = compare_similar(file1, file2)
            tmp = common_member(prev_tmp, tmp)
            #print(len(tmp))
            prev_tmp = tmp
    print(len(prev_tmp))
    ground_truth_dict[str(i)] = prev_tmp
        
        

raw_sourcecode/Apache_Maven_versions/maven-maven-3.3.4/ raw_sourcecode/Apache_Maven_versions/maven-maven-3.3.3/
raw_sourcecode/Apache_Maven_versions/maven-maven-3.3.4/ raw_sourcecode/Apache_Maven_versions/maven-maven-3.3.2/
raw_sourcecode/Apache_Maven_versions/maven-maven-3.3.4/ raw_sourcecode/Apache_Maven_versions/maven-maven-3.3.1/
raw_sourcecode/Apache_Maven_versions/maven-maven-3.3.4/ raw_sourcecode/Apache_Maven_versions/maven-maven-3.3.0/
raw_sourcecode/Apache_Maven_versions/maven-maven-3.3.4/ raw_sourcecode/Apache_Maven_versions/maven-maven-3.2.5/
raw_sourcecode/Apache_Maven_versions/maven-maven-3.3.4/ raw_sourcecode/Apache_Maven_versions/maven-maven-3.2.4/
raw_sourcecode/Apache_Maven_versions/maven-maven-3.3.4/ raw_sourcecode/Apache_Maven_versions/maven-maven-3.2.3/
raw_sourcecode/Apache_Maven_versions/maven-maven-3.3.4/ raw_sourcecode/Apache_Maven_versions/maven-maven-3.2.2/
raw_sourcecode/Apache_Maven_versions/maven-maven-3.3.4/ raw_sourcecode/Apache_Maven_versions/maven-maven

raw_sourcecode/Apache_Maven_versions/maven-maven-3.5.1/ raw_sourcecode/Apache_Maven_versions/maven-maven-3.3.7/
raw_sourcecode/Apache_Maven_versions/maven-maven-3.5.1/ raw_sourcecode/Apache_Maven_versions/maven-maven-3.3.6/
raw_sourcecode/Apache_Maven_versions/maven-maven-3.5.1/ raw_sourcecode/Apache_Maven_versions/maven-maven-3.3.5/
raw_sourcecode/Apache_Maven_versions/maven-maven-3.5.1/ raw_sourcecode/Apache_Maven_versions/maven-maven-3.3.4/
raw_sourcecode/Apache_Maven_versions/maven-maven-3.5.1/ raw_sourcecode/Apache_Maven_versions/maven-maven-3.3.3/
raw_sourcecode/Apache_Maven_versions/maven-maven-3.5.1/ raw_sourcecode/Apache_Maven_versions/maven-maven-3.3.2/
raw_sourcecode/Apache_Maven_versions/maven-maven-3.5.1/ raw_sourcecode/Apache_Maven_versions/maven-maven-3.3.1/
1800
raw_sourcecode/Apache_Maven_versions/maven-maven-3.5.2/ raw_sourcecode/Apache_Maven_versions/maven-maven-3.5.1/
raw_sourcecode/Apache_Maven_versions/maven-maven-3.5.2/ raw_sourcecode/Apache_Maven_versions/maven-

In [10]:
#C:/Users/tanji/Desktop/FIT4003/raw_sourcecode/Apache_Spark_versions/spark-1.6.0', 'C:/Users/tanji/Desktop/FIT4003/raw_sourcecode/Apache_Spark_versions/spark-2.4.4') 
#print(compare('raw_sourcecode/Apache_Spark_versions/spark-1.6.0/', 'raw_sourcecode/Apache_Spark_versions/test/spark-2.4.4'));

In [11]:
test = compare_similar('raw_sourcecode/Apache_Spark_versions/spark-1.6.0/', 'raw_sourcecode/Apache_Spark_versions/spark-2.4.4/')
test2 = compare_similar('raw_sourcecode/Apache_Spark_versions/spark-1.6.0/', 'raw_sourcecode/Apache_Spark_versions/spark-2.2.0/')
test3 = compare_similar('raw_sourcecode/Apache_Spark_versions/spark-1.6.0/', 'raw_sourcecode/Apache_Spark_versions/spark-2.2.3/')
print(len(test))
test4 = common_member(test,test2)
test5 = common_member(test4,test3)
print(len(test5))

11769
11768


In [32]:
test[0]

NameError: name 'test' is not defined

In [33]:
print(len(ground_truth_dict['10']))
print(len(ground_truth_dict))

1801
10


In [35]:
for element in ground_truth_dict['10']:
    print(element)

raw_sourcecode/Apache_Maven_versions/maven-maven-3.3.4/maven-compat\src\test\resources\inheritance-repo\t08\maven-test\jars\t08-d-1.1.jar
raw_sourcecode/Apache_Maven_versions/maven-maven-3.3.4/maven-core\src\test\resources-project-builder\single-configuration-inheritance\pom.xml
raw_sourcecode/Apache_Maven_versions/maven-maven-3.3.4/maven-core\src\test\resources-project-builder\plugin-exec-inheritance\w-merge\pom.xml
raw_sourcecode/Apache_Maven_versions/maven-maven-3.3.4/maven-core\src\test\resources-project-builder\dependency-order\w-plugin-mngt\pom.xml
raw_sourcecode/Apache_Maven_versions/maven-maven-3.3.4/maven-model-builder\src\test\resources\poms\validation\distribution-management-status.xml
raw_sourcecode/Apache_Maven_versions/maven-maven-3.3.4/maven-core\src\test\resources\org\apache\maven\extension\test-extension-repo\org\codehaus\plexus\plexus-utils\1.1\plexus-utils-1.1.pom.md5
raw_sourcecode/Apache_Maven_versions/maven-maven-3.3.4/maven-model-builder\src\test\resources\poms\v

In [37]:
for i in range(len(ground_truth_dict)):
    #print(i)
    dir_arr = []
    for element in ground_truth_dict[str(i+10)]:
        if ".java" in element:
            element = element.replace('\\', '/')
            element = element.replace('raw_sourcecode/Apache_Maven_versions/' + json_name_dict[str(i+10)].split('.json')[0] + '/', '' )
            element = element.replace('/', '\\')
            element = '\\' + element
            dir_arr.append(element)
            
    print('raw_depends/apache_maven/' + str(i+10)+ '_'+ json_name_dict[str(i+10)])
    with open('raw_depends/apache_maven/' + str(i+10)+ '_'+ json_name_dict[str(i+10)]) as f:
        spark_results = json.load(f)
        
    main_dict = {}
    main_dict_val = {}
    main_dict_counter = 0
    dict_array = []
    min_set = set()
    for element in spark_results['variables']:
        #print(element)

        ### Replace this with the initial JSON directory ###
        #print(json_name_dict[str(i+10)])
        element_tbc = element.replace('C:\\Users\\tanji\\Desktop\\FIT4003\\raw_sourcecode\\Apache_Maven_versions\\' +json_name_dict[str(i+10)].split('.json')[0] , '')
        #print(element)
        main_dict[element_tbc] = main_dict_counter
        main_dict_val[main_dict_counter] = element_tbc
        main_dict_counter += 1
 
    
    index  = 0
    var_array = []
    for value in spark_results['variables']:
        var_array.append([index, value])
        #print(index, value)
        index += 1
    
    var_df = pd.DataFrame(var_array)
    var_df.columns = ['index_val', 'name']
    
    feature_list = {}
    feature_index = 2
    for element in spark_results['cells']:
        #print(element)
        try:
            for a in element['values']:
                if a not in feature_list:
                    feature_list[a] = feature_index
                    feature_index += 1
                #print(a['Call'])


        except:
            pass
        
    feature_arr = []
    for element in spark_results['cells']:
        #print(array)
        array = [0] * (len(feature_list) + 2)
        values = dict(element['values'])
        #print(element)
        array[0] = element['src']
        array[1] = element['dest']
        for feature in feature_list:
            try:
                value = values[feature]
                array[feature_list[feature]] = value
            except:
                pass
        #print(array)
        feature_arr.append(array)
        
    feature_df = pd.DataFrame(feature_arr)
    col_names = ['src', 'dest']
    for element in feature_list:
        col_names.append(element)
    feature_df.columns = col_names
    
    feature_df['sum'] = feature_df.sum(axis=1) - feature_df['src'] - feature_df['dest']
    G = nx.Graph()
    for index, row in feature_df.iterrows():
        G.add_edge(row['src'], row['dest'], weight=row['sum'])
        
    adj_mat = nx.adjacency_matrix(G)
    adj_mat_df = pd.DataFrame(adj_mat.todense())
    np.fill_diagonal(adj_mat_df.values, adj_mat_df.values.max())
    x = adj_mat_df.values
    min_max_scaler = preprocessing.MinMaxScaler()
    x_scaled = min_max_scaler.fit_transform(x)
    adj_mat_df = pd.DataFrame(x_scaled)
    
    filename_arr = []
    for number in n_clusters:
        for affinity_type in affinity:
            for linkage_type in linkage :
                filename_1, filename_2 = alvin_god(number, affinity_type, linkage_type, dir_arr, adj_mat_df, json_name_dict[str(i+10)].split('.json')[0], main_dict)
                filename_1 = filename_1.replace('MoJo_1.2.1/', '')
                filename_2 = filename_2.replace('MoJo_1.2.1/', '')
                filename_arr.append((filename_1, filename_2))
                print(filename_1, filename_2)
    
    
    for i in range(len(filename_arr)):
    
        command = 'cd C:/Users/tanji/Desktop/FIT4003/MoJo_1.2.1 & ' + 'java MoJo ' + filename_arr[i][1] + ' ' + filename_arr[i][0] + ' >> apache_maven/apache_maven_results.txt'
        #print(command)
        #os.system('cd C:/Users/tanji/Desktop/FIT4003/MoJo_1.2.1 & java MoJo test/test2_a.rsf test/test2_b.rsf > test/test_results.txt')
        os.system(command)

raw_depends/apache_maven/10_maven-maven-3.3.4.json
apache_maven/maven-maven-3.3.4_50_euclidean_complete_b.rsf apache_maven/maven-maven-3.3.4_50_euclidean_complete_a.rsf
apache_maven/maven-maven-3.3.4_50_euclidean_average_b.rsf apache_maven/maven-maven-3.3.4_50_euclidean_average_a.rsf
apache_maven/maven-maven-3.3.4_50_euclidean_single_b.rsf apache_maven/maven-maven-3.3.4_50_euclidean_single_a.rsf
apache_maven/maven-maven-3.3.4_50_l1_complete_b.rsf apache_maven/maven-maven-3.3.4_50_l1_complete_a.rsf
apache_maven/maven-maven-3.3.4_50_l1_average_b.rsf apache_maven/maven-maven-3.3.4_50_l1_average_a.rsf
apache_maven/maven-maven-3.3.4_50_l1_single_b.rsf apache_maven/maven-maven-3.3.4_50_l1_single_a.rsf
apache_maven/maven-maven-3.3.4_50_l2_complete_b.rsf apache_maven/maven-maven-3.3.4_50_l2_complete_a.rsf
apache_maven/maven-maven-3.3.4_50_l2_average_b.rsf apache_maven/maven-maven-3.3.4_50_l2_average_a.rsf
apache_maven/maven-maven-3.3.4_50_l2_single_b.rsf apache_maven/maven-maven-3.3.4_50_l2_si

apache_maven/maven-maven-3.3.4_250_cosine_single_b.rsf apache_maven/maven-maven-3.3.4_250_cosine_single_a.rsf
apache_maven/maven-maven-3.3.4_300_euclidean_complete_b.rsf apache_maven/maven-maven-3.3.4_300_euclidean_complete_a.rsf
apache_maven/maven-maven-3.3.4_300_euclidean_average_b.rsf apache_maven/maven-maven-3.3.4_300_euclidean_average_a.rsf
apache_maven/maven-maven-3.3.4_300_euclidean_single_b.rsf apache_maven/maven-maven-3.3.4_300_euclidean_single_a.rsf
apache_maven/maven-maven-3.3.4_300_l1_complete_b.rsf apache_maven/maven-maven-3.3.4_300_l1_complete_a.rsf
apache_maven/maven-maven-3.3.4_300_l1_average_b.rsf apache_maven/maven-maven-3.3.4_300_l1_average_a.rsf
apache_maven/maven-maven-3.3.4_300_l1_single_b.rsf apache_maven/maven-maven-3.3.4_300_l1_single_a.rsf
apache_maven/maven-maven-3.3.4_300_l2_complete_b.rsf apache_maven/maven-maven-3.3.4_300_l2_complete_a.rsf
apache_maven/maven-maven-3.3.4_300_l2_average_b.rsf apache_maven/maven-maven-3.3.4_300_l2_average_a.rsf
apache_maven/m

apache_maven/maven-maven-3.3.5_200_cosine_average_b.rsf apache_maven/maven-maven-3.3.5_200_cosine_average_a.rsf
apache_maven/maven-maven-3.3.5_200_cosine_single_b.rsf apache_maven/maven-maven-3.3.5_200_cosine_single_a.rsf
apache_maven/maven-maven-3.3.5_250_euclidean_complete_b.rsf apache_maven/maven-maven-3.3.5_250_euclidean_complete_a.rsf
apache_maven/maven-maven-3.3.5_250_euclidean_average_b.rsf apache_maven/maven-maven-3.3.5_250_euclidean_average_a.rsf
apache_maven/maven-maven-3.3.5_250_euclidean_single_b.rsf apache_maven/maven-maven-3.3.5_250_euclidean_single_a.rsf
apache_maven/maven-maven-3.3.5_250_l1_complete_b.rsf apache_maven/maven-maven-3.3.5_250_l1_complete_a.rsf
apache_maven/maven-maven-3.3.5_250_l1_average_b.rsf apache_maven/maven-maven-3.3.5_250_l1_average_a.rsf
apache_maven/maven-maven-3.3.5_250_l1_single_b.rsf apache_maven/maven-maven-3.3.5_250_l1_single_a.rsf
apache_maven/maven-maven-3.3.5_250_l2_complete_b.rsf apache_maven/maven-maven-3.3.5_250_l2_complete_a.rsf
apache

apache_maven/maven-maven-3.3.6_150_cosine_complete_b.rsf apache_maven/maven-maven-3.3.6_150_cosine_complete_a.rsf
apache_maven/maven-maven-3.3.6_150_cosine_average_b.rsf apache_maven/maven-maven-3.3.6_150_cosine_average_a.rsf
apache_maven/maven-maven-3.3.6_150_cosine_single_b.rsf apache_maven/maven-maven-3.3.6_150_cosine_single_a.rsf
apache_maven/maven-maven-3.3.6_200_euclidean_complete_b.rsf apache_maven/maven-maven-3.3.6_200_euclidean_complete_a.rsf
apache_maven/maven-maven-3.3.6_200_euclidean_average_b.rsf apache_maven/maven-maven-3.3.6_200_euclidean_average_a.rsf
apache_maven/maven-maven-3.3.6_200_euclidean_single_b.rsf apache_maven/maven-maven-3.3.6_200_euclidean_single_a.rsf
apache_maven/maven-maven-3.3.6_200_l1_complete_b.rsf apache_maven/maven-maven-3.3.6_200_l1_complete_a.rsf
apache_maven/maven-maven-3.3.6_200_l1_average_b.rsf apache_maven/maven-maven-3.3.6_200_l1_average_a.rsf
apache_maven/maven-maven-3.3.6_200_l1_single_b.rsf apache_maven/maven-maven-3.3.6_200_l1_single_a.rs

apache_maven/maven-maven-3.3.7_100_manhattan_single_b.rsf apache_maven/maven-maven-3.3.7_100_manhattan_single_a.rsf
apache_maven/maven-maven-3.3.7_100_cosine_complete_b.rsf apache_maven/maven-maven-3.3.7_100_cosine_complete_a.rsf
apache_maven/maven-maven-3.3.7_100_cosine_average_b.rsf apache_maven/maven-maven-3.3.7_100_cosine_average_a.rsf
apache_maven/maven-maven-3.3.7_100_cosine_single_b.rsf apache_maven/maven-maven-3.3.7_100_cosine_single_a.rsf
apache_maven/maven-maven-3.3.7_150_euclidean_complete_b.rsf apache_maven/maven-maven-3.3.7_150_euclidean_complete_a.rsf
apache_maven/maven-maven-3.3.7_150_euclidean_average_b.rsf apache_maven/maven-maven-3.3.7_150_euclidean_average_a.rsf
apache_maven/maven-maven-3.3.7_150_euclidean_single_b.rsf apache_maven/maven-maven-3.3.7_150_euclidean_single_a.rsf
apache_maven/maven-maven-3.3.7_150_l1_complete_b.rsf apache_maven/maven-maven-3.3.7_150_l1_complete_a.rsf
apache_maven/maven-maven-3.3.7_150_l1_average_b.rsf apache_maven/maven-maven-3.3.7_150_l

...

apache_maven/maven-maven-3.5.0_200_l2_average_b.rsf apache_maven/maven-maven-3.5.0_200_l2_average_a.rsf
apache_maven/maven-maven-3.5.0_200_l2_single_b.rsf apache_maven/maven-maven-3.5.0_200_l2_single_a.rsf
apache_maven/maven-maven-3.5.0_200_manhattan_complete_b.rsf apache_maven/maven-maven-3.5.0_200_manhattan_complete_a.rsf
apache_maven/maven-maven-3.5.0_200_manhattan_average_b.rsf apache_maven/maven-maven-3.5.0_200_manhattan_average_a.rsf
apache_maven/maven-maven-3.5.0_200_manhattan_single_b.rsf apache_maven/maven-maven-3.5.0_200_manhattan_single_a.rsf
apache_maven/maven-maven-3.5.0_200_cosine_complete_b.rsf apache_maven/maven-maven-3.5.0_200_cosine_complete_a.rsf
apache_maven/maven-maven-3.5.0_200_cosine_average_b.rsf apache_maven/maven-maven-3.5.0_200_cosine_average_a.rsf
apache_maven/maven-maven-3.5.0_200_cosine_single_b.rsf apache_maven/maven-maven-3.5.0_200_cosine_single_a.rsf
apache_maven/maven-maven-3.5.0_250_euclidean_complete_b.rsf apache_maven/maven-maven-3.5.0_250_euclidean

apache_maven/maven-maven-3.5.1_150_l2_complete_b.rsf apache_maven/maven-maven-3.5.1_150_l2_complete_a.rsf
apache_maven/maven-maven-3.5.1_150_l2_average_b.rsf apache_maven/maven-maven-3.5.1_150_l2_average_a.rsf
apache_maven/maven-maven-3.5.1_150_l2_single_b.rsf apache_maven/maven-maven-3.5.1_150_l2_single_a.rsf
apache_maven/maven-maven-3.5.1_150_manhattan_complete_b.rsf apache_maven/maven-maven-3.5.1_150_manhattan_complete_a.rsf
apache_maven/maven-maven-3.5.1_150_manhattan_average_b.rsf apache_maven/maven-maven-3.5.1_150_manhattan_average_a.rsf
apache_maven/maven-maven-3.5.1_150_manhattan_single_b.rsf apache_maven/maven-maven-3.5.1_150_manhattan_single_a.rsf
apache_maven/maven-maven-3.5.1_150_cosine_complete_b.rsf apache_maven/maven-maven-3.5.1_150_cosine_complete_a.rsf
apache_maven/maven-maven-3.5.1_150_cosine_average_b.rsf apache_maven/maven-maven-3.5.1_150_cosine_average_a.rsf
apache_maven/maven-maven-3.5.1_150_cosine_single_b.rsf apache_maven/maven-maven-3.5.1_150_cosine_single_a.rs

apache_maven/maven-maven-3.5.2_100_l1_single_b.rsf apache_maven/maven-maven-3.5.2_100_l1_single_a.rsf
apache_maven/maven-maven-3.5.2_100_l2_complete_b.rsf apache_maven/maven-maven-3.5.2_100_l2_complete_a.rsf
apache_maven/maven-maven-3.5.2_100_l2_average_b.rsf apache_maven/maven-maven-3.5.2_100_l2_average_a.rsf
apache_maven/maven-maven-3.5.2_100_l2_single_b.rsf apache_maven/maven-maven-3.5.2_100_l2_single_a.rsf
apache_maven/maven-maven-3.5.2_100_manhattan_complete_b.rsf apache_maven/maven-maven-3.5.2_100_manhattan_complete_a.rsf
apache_maven/maven-maven-3.5.2_100_manhattan_average_b.rsf apache_maven/maven-maven-3.5.2_100_manhattan_average_a.rsf
apache_maven/maven-maven-3.5.2_100_manhattan_single_b.rsf apache_maven/maven-maven-3.5.2_100_manhattan_single_a.rsf
apache_maven/maven-maven-3.5.2_100_cosine_complete_b.rsf apache_maven/maven-maven-3.5.2_100_cosine_complete_a.rsf
apache_maven/maven-maven-3.5.2_100_cosine_average_b.rsf apache_maven/maven-maven-3.5.2_100_cosine_average_a.rsf
apache

apache_maven/maven-maven-3.5.3_50_l1_average_b.rsf apache_maven/maven-maven-3.5.3_50_l1_average_a.rsf
apache_maven/maven-maven-3.5.3_50_l1_single_b.rsf apache_maven/maven-maven-3.5.3_50_l1_single_a.rsf
apache_maven/maven-maven-3.5.3_50_l2_complete_b.rsf apache_maven/maven-maven-3.5.3_50_l2_complete_a.rsf
apache_maven/maven-maven-3.5.3_50_l2_average_b.rsf apache_maven/maven-maven-3.5.3_50_l2_average_a.rsf
apache_maven/maven-maven-3.5.3_50_l2_single_b.rsf apache_maven/maven-maven-3.5.3_50_l2_single_a.rsf
apache_maven/maven-maven-3.5.3_50_manhattan_complete_b.rsf apache_maven/maven-maven-3.5.3_50_manhattan_complete_a.rsf
apache_maven/maven-maven-3.5.3_50_manhattan_average_b.rsf apache_maven/maven-maven-3.5.3_50_manhattan_average_a.rsf
apache_maven/maven-maven-3.5.3_50_manhattan_single_b.rsf apache_maven/maven-maven-3.5.3_50_manhattan_single_a.rsf
apache_maven/maven-maven-3.5.3_50_cosine_complete_b.rsf apache_maven/maven-maven-3.5.3_50_cosine_complete_a.rsf
apache_maven/maven-maven-3.5.3_5

apache_maven/maven-maven-3.5.3_300_l1_complete_b.rsf apache_maven/maven-maven-3.5.3_300_l1_complete_a.rsf
apache_maven/maven-maven-3.5.3_300_l1_average_b.rsf apache_maven/maven-maven-3.5.3_300_l1_average_a.rsf
apache_maven/maven-maven-3.5.3_300_l1_single_b.rsf apache_maven/maven-maven-3.5.3_300_l1_single_a.rsf
apache_maven/maven-maven-3.5.3_300_l2_complete_b.rsf apache_maven/maven-maven-3.5.3_300_l2_complete_a.rsf
apache_maven/maven-maven-3.5.3_300_l2_average_b.rsf apache_maven/maven-maven-3.5.3_300_l2_average_a.rsf
apache_maven/maven-maven-3.5.3_300_l2_single_b.rsf apache_maven/maven-maven-3.5.3_300_l2_single_a.rsf
apache_maven/maven-maven-3.5.3_300_manhattan_complete_b.rsf apache_maven/maven-maven-3.5.3_300_manhattan_complete_a.rsf
apache_maven/maven-maven-3.5.3_300_manhattan_average_b.rsf apache_maven/maven-maven-3.5.3_300_manhattan_average_a.rsf
apache_maven/maven-maven-3.5.3_300_manhattan_single_b.rsf apache_maven/maven-maven-3.5.3_300_manhattan_single_a.rsf
apache_maven/maven-mav

In [29]:
dir_arr

['\\core\\src\\main\\java\\org\\apache\\spark\\util\\collection\\unsafe\\sort\\UnsafeSorterSpillWriter.java',
 '\\examples\\src\\main\\java\\org\\apache\\spark\\examples\\ml\\JavaDecisionTreeRegressionExample.java',
 '\\launcher\\src\\main\\java\\org\\apache\\spark\\launcher\\CommandBuilderUtils.java',
 '\\examples\\src\\main\\java\\org\\apache\\spark\\examples\\ml\\JavaNGramExample.java',
 '\\launcher\\src\\main\\java\\org\\apache\\spark\\launcher\\ChildProcAppHandle.java',
 '\\sql\\catalyst\\src\\main\\java\\org\\apache\\spark\\sql\\catalyst\\expressions\\codegen\\UnsafeArrayWriter.java',
 '\\graphx\\src\\main\\scala\\org\\apache\\spark\\graphx\\util\\package-info.java',
 '\\core\\src\\main\\java\\org\\apache\\spark\\shuffle\\sort\\UnsafeShuffleWriter.java',
 '\\sql\\hive\\src\\test\\java\\org\\apache\\spark\\sql\\hive\\execution\\UDFIntegerToString.java',
 '\\streaming\\src\\test\\java\\org\\apache\\spark\\streaming\\JavaAPISuite.java',
 '\\mllib\\src\\test\\java\\org\\apache\\spark

In [30]:
main_dict

{'\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\TransportContext.java': 0,
 '\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\buffer\\FileSegmentManagedBuffer.java': 1,
 '\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\buffer\\ManagedBuffer.java': 2,
 '\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\buffer\\NettyManagedBuffer.java': 3,
 '\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\buffer\\NioManagedBuffer.java': 4,
 '\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\client\\ChunkFetchFailureException.java': 5,
 '\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\client\\ChunkReceivedCallback.java': 6,
 '\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\client\\RpcResponseCallback.java': 7,
 '\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\client\\StreamCallback.java': 8,
 '\\

In [31]:
main_dict_val

{0: '\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\TransportContext.java',
 1: '\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\buffer\\FileSegmentManagedBuffer.java',
 2: '\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\buffer\\ManagedBuffer.java',
 3: '\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\buffer\\NettyManagedBuffer.java',
 4: '\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\buffer\\NioManagedBuffer.java',
 5: '\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\client\\ChunkFetchFailureException.java',
 6: '\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\client\\ChunkReceivedCallback.java',
 7: '\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\client\\RpcResponseCallback.java',
 8: '\\common\\network-common\\src\\main\\java\\org\\apache\\spark\\network\\client\\StreamCallback.java',
 9: 

In [ ]:
filename_arr

In [ ]:
vectorizer = TfidfVectorizer(stop_words='english')
X = vectorizer.fit_transform(dir_arr)

true_k = 300
model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
model.fit(X)

#print("Top terms per cluster:")
order_centroids = model.cluster_centers_.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

In [ ]:
Y = vectorizer.transform(['\\mllib\\src\\test\\java\\org\\apache\\spark\\ml\\classification\\JavaNaiveBayesSuite.java'])
prediction = model.predict(Y)
print(prediction)

Y = vectorizer.transform(['\\mllib\\src\\test\\java\\org\\apache\\spark\\mllib\\classification\\JavaNaiveBayesSuite.java'])
prediction = model.predict(Y)
print(prediction)

In [4]:
command = 'cd C:/Users/tanji/Desktop/FIT4003/ & ' + 'java -jar ck-0.3.3-SNAPSHOT-jar-with-dependencies.jar ' + dir_arr[-1]
print(command)
#os.system('cd C:/Users/tanji/Desktop/FIT4003/MoJo_1.2.1 & java MoJo test/test2_a.rsf test/test2_b.rsf > test/test_results.txt')
os.system(command)
import shutil
os.mkdir('Final_Results/' + project_name)
os.rename('class.csv', project_name + '_class.csv')
os.rename('field.csv', project_name + '_field.csv')
os.rename('method.csv', project_name + '_method.csv')
shutil.copy('MoJo_1.2.1/' + project_name + '/' + project_name + '_results.txt' , 'Final_Results/' + project_name)
shutil.copy(project_name + '_class.csv' , 'Final_Results/' + project_name)
shutil.copy(project_name + '_field.csv' , 'Final_Results/' + project_name)
shutil.copy(project_name + '_method.csv' , 'Final_Results/' + project_name)
os.remove(project_name + '_class.csv')
os.remove(project_name + '_field.csv')
os.remove(project_name + '_method.csv')

cd C:/Users/tanji/Desktop/FIT4003/ & java -jar ck-0.3.3-SNAPSHOT-jar-with-dependencies.jar raw_sourcecode\Apache_Maven_versions\maven-maven-3.6.2


In [ ]:
release_arr = []
project_main_name = 'apache-maven'

os.mkdir('C:/Users/tanji/Desktop/FIT4003/raw_sourcecode/' + project_main_name)
project_name = 'spark-'
for index, row in ver.iterrows():
    #print(index)
    #print('lol')
    #print(row['release_tag'])
    release_arr.append((row['release_tag'], row['version_name']))

for element in release_arr:
    command = 'cd C:/Users/tanji/Desktop/FIT4003/raw_sourcecode/' + project_main_name + ' & mkdir ' + project_name + element[1]
    print(command)
    os.system(command)
    
    command = 'git clone https://github.com/apache/spark ' +  'C:/Users/tanji/Desktop/FIT4003/raw_sourcecode/' +project_main_name + '/' + project_name + element[1]
    print(command)
    os.system(command)
    
    command = 'cd C:/Users/tanji/Desktop/FIT4003/raw_sourcecode/' + project_name + element[1] + ' git checkout ' + element[0]
    print(command)
    os.system(command)

rootdir = 'raw_sourcecode'
dir_arr = []
depth=2

for root, dirs, files in os.walk(rootdir):
    if root.count(os.sep) == depth and "apache-spark" in str(root):
        print(root)
        dir_arr.append(root)
        
        
        
project_name = 'apache-spark'
os.mkdir('Final_Results/' + project_name)

for i in range(11,21):
    print(dir_arr[i])
    command = 'cd C:/Users/tanji/Desktop/FIT4003/ & ' + 'java -jar ck-0.3.3-SNAPSHOT-jar-with-dependencies.jar ' + dir_arr[i]
    print(command)
    os.system(command)
    import shutil
    #os.mkdir('Final_Results/' + project_name)
    os.rename('class.csv', project_name + '-' + dir_arr[i].split('\\')[-1] +  '_class.csv')
    os.rename('field.csv', project_name + '-' + dir_arr[i].split('\\')[-1] + '_field.csv')
    os.rename('method.csv', project_name + '-' + dir_arr[i].split('\\')[-1] + '_method.csv')
    #shutil.copy('MoJo_1.2.1/' + project_name + '/' + project_name + '_results.txt' , 'Final_Results/' + project_name)
    shutil.copy(project_name + '-' + dir_arr[i].split('\\')[-1] +  '_class.csv' , 'Final_Results/' + project_name)
    shutil.copy(project_name + '-' + dir_arr[i].split('\\')[-1] + '_field.csv' , 'Final_Results/' + project_name)
    shutil.copy(project_name + '-' + dir_arr[i].split('\\')[-1] + '_method.csv' , 'Final_Results/' + project_name)
    os.remove(project_name + '-' + dir_arr[i].split('\\')[-1] +  '_class.csv')
    os.remove(project_name + '-' + dir_arr[i].split('\\')[-1] + '_field.csv')
    os.remove(project_name + '-' + dir_arr[i].split('\\')[-1] + '_method.csv')